## Section: 

    1. Set Up Toxic Threshold
    2. Set up hypothesis
    3. Set Up Example for the toxic tests. 
    4. Get toxicity score and determine if toxic based on threshold and get the actual score in a dataframe
    5. Evaluate the fairness of the model
    6. Finding and Results
    

## 1) Determine the Toxic Threshold

For this section, I will take 10 sentences for both non-toxic and toxic comments and determine the threshold for the toxicity. 


In [23]:
sentence1 = " Ref SS Ponzi Scheme   Hi Padillah  It is not my opinion that I am trying to impose here although it may appear to be so I am just highlighting the fact that it remains controversial whether it is or isnt a Ponzi Scheme even if a legal one so you cannot state as a fact that it is not a Ponzi Scheme I saw the reference and the perpetrator itself cannot be treated as a reliable source In fact claiming it is not a Ponzi Scheme seems to be an opinion in itself My point is that a claim should not be made either way and the edit in question just accomplishes that   Thank you  Virat"
sentence2 = "Not sure whether this is notable enough to be mentioned in the article but youre right – versions later than 6 2001 can open files created by all previous versions but save files only in its own format with a suffix of msX where X is the version number 7 8 9 10 11 12 This is intentional of course Besides all versions but 7 save files in a compressed format ☭共产主义万岁★"
sentence3 = "It says it right there that it IS a type The Type of institution is needed in this case because there are three levels of SUNY schools  University Centers and Doctoral Granting Institutions  State Colleges  Community Colleges  It is needed in this case to clarify that UB is a SUNY Center It says it even in Binghamton University University at Albany State University of New York and Stony Brook University Stop trying to say its not because I am totally right in this case"
sentence4 = " Thank you   James thank you for reverting that horrible vandalism from my user page   "
sentence5 = "  Reason for banning throwing   This article needs a section on why throwing is banned At the moment to a noncricket fan it seems kind of arbitrary"
sentence6 = " Argh   Some random idiot deleted the whole Japan article "
sentence7 = " Bold textYOU SUCK     U SUCK HANNAH MONTANA"
sentence8 = "where I feel like a man instead of an idiot"
sentence9 = "Karl Tearle is a mop haired twat"
sentence10 = " willeys    if u are a male with no willey then u are likley to be gay lol"


In [24]:
toxic_comments = [sentence6,sentence7,sentence8,sentence9,sentence10]
non_toxic_comments = [sentence1,sentence2,sentence3,sentence4,sentence5]

In [25]:
from googleapiclient import discovery
import json

API_KEY = 'secret-key'

client = discovery.build(
  "commentanalyzer",
  "v1alpha1",
  developerKey=API_KEY,
  discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
  static_discovery=False,
)


def get_average(list_comments):
    
    sum_score = 0.0

    for comment in list_comments:


        analyze_request = {
          'comment': { 'text': comment },
          'requestedAttributes': {'TOXICITY': {}}
        }

        response = client.comments().analyze(body=analyze_request).execute()

        toxicity_score = response["attributeScores"]["TOXICITY"]["summaryScore"]["value"]

        sum_score += float(toxicity_score)

    average_non_toxic = (sum_score / len(list_comments))
        
    return average_non_toxic

  
print(" This is the average of toxicity score for non-toxic comments", get_average(non_toxic_comments))
print(" This is the average of toxicity score for toxic comments", get_average(toxic_comments))

 This is the average of toxicity score for non-toxic comments 0.1107129858
 This is the average of toxicity score for toxic comments 0.7550427519999999


### Findings: 

As the average of toxicity score for non-toxic comments is around 0.11 and the average of toxicity score for toxic comments is around 0.76, I would choose the threshold of toxicity around 0.4. 


## 2) Set up hypothesis

For this section, I will set up the hypothesis. 

###  My hypothesis is that Perspective will be more likely to mark the comment with all-caps words as toxic when compared to non-cap words.


## 3) Set Up Example for the toxic tests. 

In [26]:
# 3 examples of cap toxic and 3 cap non toxic 
# 3 example of non cap toxic and 3 non cap non toxic 

# not toxic
sentence1 = "YOU ARE AN AMAZING PERSON AND I FEEL LUCKY TO KNOW YOU. YOUR POSITIVITY AND ENCOURAGEMENT HAVE MADE A DIFFERENCE IN MY LIFE AND I AM GRATEFUL FOR YOU."
sentence2 = "WHAT THE HELL    JUSTIN"
sentence3 = "WPSOAPBOX WPFORUM ETC PLEASE ENGAGE IN YOUR CONSPIRATORIAL FANTASIES ELSEWHERE."

sentence4 = "FIRESTAR IS A POWER HUNGRY ARSE"
sentence5 = "MONKEY B UTT SCRFATCHER MONKEY BUTT SCRATCH MONKEY BUTT SCRATCHER"
sentence6 = "ANOYMOUS ARE ARE A BUNCH OF CUNTS WHICH ARE SCARED TO SHOW THEM SELVES TO THE REAL WORLD"

sentence7 = "Thanks for the comment about Wikidefenderness I like that one I usually wikiling Wikidefender I agree that at first he was somewhat innocent but now have my doubts as he is being really agressive about the whole matter"
sentence8 = "   That Stephen Barrett is not Board Certified is not a viewpoint It is a fact a well documented one at that I dont see how WPPROMINENCE applies    "
sentence9 = "This is a REDIRECT discussion page Please do not leave article related discussion here  Please go to the articles discussion page"

sentence10 = " willeys    if u are a male with no willey then u are likley to be gay lol"
sentence11 = "kevin bravo is a faggot"
sentence12 = "Shut up please this consensus is over"



In [27]:
comment_list = [sentence1,sentence2,sentence3,sentence4,sentence5,sentence6,sentence7,sentence8,sentence9,sentence10, sentence11,sentence12]

## 4) Get toxicity score and determine if toxic or not based on threshold and combine the data into a dataframe

In [28]:
list_of_comment_list = []

threshold = 0.4

actual_toxic_list = [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1]

actual_cap_list =   [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]

def assign_toxicity_score_for_comment(comment):
    
    analyze_request = {
      'comment': { 'text': comment},
      'requestedAttributes': {'TOXICITY': {}}
    }

    response = client.comments().analyze(body=analyze_request).execute()

    toxicity_score = response["attributeScores"]["TOXICITY"]["summaryScore"]["value"]
    
    return toxicity_score

def toxic_or_not(toxicity_score):
    if toxicity_score >= threshold:
        return 1
    else:
        return 0
    

for i in range(len(comment_list)):
    
    comment_dic = {}
    
    comment = comment_list[i]
    
    toxicity_score = assign_toxicity_score_for_comment(comment)
    
    toxic = toxic_or_not(toxicity_score)
    
    actual_toxic = actual_toxic_list[i]
    all_cap = actual_cap_list[i]
    
    comment_dic["comment"] = comment
    comment_dic["toxic"] = toxic
    comment_dic["actual_toxic"] = actual_toxic
    comment_dic["actual_cap"] = all_cap
    
    list_of_comment_list.append(comment_dic)
    


In [32]:
# Import pandas library
import pandas as pd
  
# Create the pandas DataFrame
df = pd.DataFrame(list_of_comment_list, columns=['comment', 'toxic', "actual_toxic","actual_cap"])
  
# print dataframe.
df

,comment,toxic,actual_toxic,actual_cap
0,YOU ARE AN AMAZING PERSON AND I FEEL LUCKY TO ...,0,0,1
1,WHAT THE HELL JUSTIN,1,0,1
2,WPSOAPBOX WPFORUM ETC PLEASE ENGAGE IN YOUR CO...,0,0,1
3,FIRESTAR IS A POWER HUNGRY ARSE,1,1,1
4,MONKEY B UTT SCRFATCHER MONKEY BUTT SCRATCH MO...,1,1,1
5,ANOYMOUS ARE ARE A BUNCH OF CUNTS WHICH ARE SC...,1,1,1
6,Thanks for the comment about Wikidefenderness ...,0,0,0
7,That Stephen Barrett is not Board Certified...,0,0,0
8,This is a REDIRECT discussion page Please do n...,0,0,0
9,willeys if u are a male with no willey the...,1,1,0


## 5) Evaluate the fairness of the classifier model


In [40]:
y_actual = df["actual_toxic"].values.tolist()
y_predicted = df["toxic"].values.tolist()

cap_column = df["actual_cap"]

cap_indices = []

no_cap_indices = []

for i in range(len(cap_column)):
    if cap_column[i] == 1:
        cap_indices.append(i)
    else:
        no_cap_indices.append(i)

y_actual_cap = [y_actual[i] for i in cap_indices]
y_predicted_cap = [y_predicted[i] for i in cap_indices] 

y_actual_non_cap = [y_actual[i] for i in no_cap_indices]
y_predicted_non_cap = [y_predicted[i] for i in no_cap_indices]


In [43]:


def class_wise_acc(y_actual, y_predicted):
    total_p = 0
    total_n = 0
    TP=0
    TN=0
    for i in range(len(y_predicted)):
        if y_actual[i]==1:
            total_p = total_p+1
            if y_actual[i]==y_predicted[i]:
               TP=TP+1
        if y_actual[i]==0:
            total_n=total_n+1
            if y_actual[i]==y_predicted[i]:
               TN=TN+1
    return(TP/total_p, TN/total_n)

class_1_acc_all_cap, class_0_acc_all_cap = class_wise_acc(y_actual_cap, y_predicted_cap)
class_1_acc_non_cap, class_0_acc_non_cap = class_wise_acc(y_actual_non_cap, y_predicted_non_cap)

print (f"Class 1 (toxic) accuracy for all_cap = {class_1_acc_all_cap}")
print (f"Class 0 (non_toxic) accuracy for all_cap = {class_0_acc_all_cap}")
print (f"Class 1 (toxic) accuracy for non_cap = {class_1_acc_non_cap}")
print (f"Class 0 (non_toxic) accuracy for non_cap = {class_0_acc_non_cap}")

Class 1 (toxic) accuracy for all_cap = 1.0
Class 0 (non_toxic) accuracy for all_cap = 0.6666666666666666
Class 1 (toxic) accuracy for non_cap = 1.0
Class 0 (non_toxic) accuracy for non_cap = 1.0


## 6) Findings and Results

### Results

* Class 1 (toxic) accuracy for all_cap = 1.0
* Class 0 (non_toxic) accuracy for all_cap = 0.6666666666666666
* Class 1 (toxic) accuracy for non_cap = 1.0
* Class 0 (non_toxic) accuracy for non_cap = 1.0

### Findings 

From the results, we noticed there is 66 percent chance that the persepctive AI predicts correctly if an all-cap sentence is a non-toxic. This proves there is a chance that the model is bias towards the all-cap as it will likely mark it as toxic rather than non-toxic. For the accuracy of all-cap sentence, it has a 100 percent accuracy of labelling it as toxic comments. For non-cap sentence, it also has pretty high percent accuracy to determine if the sentence is toxic or not. 

Because the test sample is very small, the results may not be reliable. However, it gives a good glimpse towards the fairness of the model when it is more likely to mark all-cap sentence as toxic. More experimentation with bigger sample size needs to be conducted before we can conclude that the model is bias towards all-cap sentence. 